In [1]:
import math

import torch.nn as nn

/data/heerak/workspace/venv3.8/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class LunaBlock(nn.Module):
    
    def __init__(self, in_channels, conv_channels):
        super().__init__()
        self.conv1 = nn.Conv3d(
            in_channels, conv_channels, kernel_size=3, padding=1, bias=True,
        )
        self.relu1 = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv3d(
            conv_channels, conv_channels, kernel_size=3, padding=1, bias=True,
        )
        self.relu2 = nn.ReLU(inplace=True)
        
        self.maxpool = nn.MaxPool3d(2, 2)
        
    def forward(self, input_batch):
        block_out = self.conv1(input_batch)
        block_out = self.relu1(block_out)
        block_out = self.conv2(block_out)
        block_out = self.relu2(block_out)
        
        return self.maxpool(block_out)

In [3]:
class LunaModel(nn.Module):
    
    def __init__(self, in_channels=1, conv_channels=8):
        super().__init__()
        
        self.tail_backbone = nn.BachNorm3d(1)  # 테일
        
        # 백본
        self.block1 = LunaBlock(in_channels, conv_channels)
        self.block2 = LunaBlock(conv_channels, conv_channels * 2)
        self.block3 = LunaBlock(conv_channels * 2, conv_channels * 4)
        self.block4 = LunaBlock(conv_channels * 4, conv_channels * 8)
        
        # 헤드
        self.head_linear = nn.Linear(1152, 2)
        self.head_softmax = nn.Softmax(dim=1)
        
    def _init_weights(self):
        for m in self.modules():
            if type(m) in {nn.Linear,
                           nn.Conv3d,
                          }:
                nn.init.kaming_normal_(
                    m.weight.data, a=0, mode='fan_out', nonlinearity='relu',
                )
                if m.bias is not none:
                    fan_in, fan_out = nn.init._calculate_fan_in_and_fan_out(m.weight.data)
                    bound = 1 / math.sqrt(fan_out)
                    nn.init.normal_(m.bias, -bound, bound)
        
    def forward(self, input_batch):
        bn_output = self.tail_backbone(input_batch)
        
        block_out = self.block1(bn_output)
        block_out = self.block2(block_out)
        block_out = self.block3(block_out)
        block_out = self.block4(block_out)
        
        conv_flat = block_out.view(block_out.size(0),  # 배치 크기
                                   -1)
        
        linear_output = self.head_linear(conv_float)
        
        return linear_output, self.head_softmax(lineaer_output)